### **Librerías**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [3]:
import h2o
from h2o.automl import H2OAutoML

## **Datos**

In [4]:
data = load_breast_cancer()

In [5]:
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.DataFrame({'Target': data.target})

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [7]:
train_df = pd.concat([X_train, y_train], axis=1)
train_df.sample()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Target
424,9.742,19.12,61.93,289.7,0.1075,0.08333,0.008934,0.01967,0.2538,0.07029,...,23.17,71.79,380.9,0.1398,0.1352,0.02085,0.04589,0.3196,0.08009,1


In [8]:
test_df = pd.concat([X_test, y_test], axis=1)
test_df.sample()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Target
464,13.17,18.22,84.28,537.3,0.07466,0.05994,0.04859,0.0287,0.1454,0.05549,...,23.89,95.1,687.6,0.1282,0.1965,0.1876,0.1045,0.2235,0.06925,1


## **Modelo**

In [9]:
# Inicializamos h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu11.43+55-CA (build 11.0.9.1+1-LTS, mixed mode)
  Starting server from C:\Users\jmart\Documents\Proyectos\01. Data Science\00. Generalidades\good_practices_DS\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\jmart\AppData\Local\Temp\tmp2s35uw8e
  JVM stdout: C:\Users\jmart\AppData\Local\Temp\tmp2s35uw8e\h2o_jmart_started_from_python.out
  JVM stderr: C:\Users\jmart\AppData\Local\Temp\tmp2s35uw8e\h2o_jmart_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Bogota
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_jmart_h8honh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.926 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [10]:
h2o_train = h2o.H2OFrame(train_df)
h2o_test = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
h2o_train['Target'] = h2o_train['Target'].asfactor()
h2o_test['Target'] = h2o_test['Target'].asfactor()

In [12]:
X_columns = h2o_train.drop('Target', axis=1).columns
y_columns = 'Target'

In [13]:
model = H2OAutoML(
    max_runtime_secs=60,
    seed=13
)

In [14]:
model.train(
    x=X_columns,
    y=y_columns, 
    training_frame=h2o_train
)

AutoML progress: |


17:04:49.926: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/5
# GBM base models (used / total),1/1
# GLM base models (used / total),1/1
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [15]:
leaderboard = model.leaderboard
leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_4_AutoML_1_20231230_170449,0.995601,0.0760852,0.996951,0.0281036,0.143633,0.0206306
StackedEnsemble_AllModels_1_AutoML_1_20231230_170449,0.995479,0.0780943,0.996955,0.0291524,0.1461,0.0213453
GBM_4_AutoML_1_20231230_170449,0.995398,0.0858732,0.997066,0.0291524,0.155693,0.0242404
StackedEnsemble_BestOfFamily_2_AutoML_1_20231230_170449,0.995092,0.0790819,0.996558,0.0231651,0.143694,0.0206481
GBM_grid_1_AutoML_1_20231230_170449_model_6,0.995011,0.0860014,0.996724,0.0324312,0.157083,0.024675
GLM_1_AutoML_1_20231230_170449,0.994888,0.0816401,0.996166,0.0242139,0.145972,0.0213079
GBM_grid_1_AutoML_1_20231230_170449_model_5,0.994848,0.0936099,0.996726,0.0348342,0.163909,0.0268661
StackedEnsemble_AllModels_2_AutoML_1_20231230_170449,0.994481,0.0774334,0.995761,0.0203242,0.139949,0.0195856
StackedEnsemble_BestOfFamily_3_AutoML_1_20231230_170449,0.99442,0.0837638,0.995956,0.0231651,0.147591,0.0217832
DeepLearning_grid_3_AutoML_1_20231230_170449_model_1,0.9944,0.095516,0.996167,0.0306391,0.158103,0.0249966


In [ ]:
# h2o.shutdown()

H2O session _sid_a7ed closed.


C:\Users\jmart\AppData\Local\Temp\ipykernel_19652\2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
